# Section B - Working with DataFrames and SQL

Work with a large dataset in CSV format for Los Angeles Parking Citations dataset (parking-citations.csv).

In [1]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

from pprint import pprint
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",3)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .appName("Siwei_Fu_A3")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

## B.1 Load the CSV file from HDFS and call show() to verify the data

In [2]:
# Loading the csv data
parking_citations = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.113:9000/parking-citations.csv')\
    .cache()

# Call show() to verify the data is loaded correctly
parking_citations.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+
|   1103341116|2015-12-21T00:00:00|      1251|    null|       null|            CA|           200304|null|HOND|        PA|   GY|     13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|         50|    99999|    99999|
|   1103700150|2015-12-21T00:00:00|      1435|    null|       null|         

## B.2 Print the schema for the DataFrame

In [3]:
parking_citations.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



## B.3 Count the number of rows in the CSV file.

In [4]:
# The number of rows in the CSV file is 9257460.

num_line = parking_citations.count()
print(f'The number of rows in the CSV file is {num_line}.')

The number of rows in the CSV file is 9257460.


## B.4 Count the number of partitions in the underlying RDD.

In [5]:
# Tthe number of partitions in the underlying RDD is 10.

num_par = parking_citations.rdd.getNumPartitions()
print(f'The number of partitions in the underlying rdd {num_par}.')

The number of partitions in the underlying rdd 10.


## B.5 Drop the columns VIN, Latitude and Longitude.

In [6]:
df_drop = parking_citations.drop('VIN', 'Latitude', 'Longitude')
print(f'The head before drop the columns is: \n{parking_citations.head()}\n')
print(f'The head after drop the columns is: \n{df_drop.head()}')

The head before drop the columns is: 
Row(Ticket number='1103341116', Issue Date='2015-12-21T00:00:00', Issue time='1251', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='200304', VIN=None, Make='HOND', Body Style='PA', Color='GY', Location='13147 WELBY WAY', Route='01521', Agency='1', Violation code='4000A1', Violation Description='NO EVIDENCE OF REG', Fine amount='50', Latitude='99999', Longitude='99999')

The head after drop the columns is: 
Row(Ticket number='1103341116', Issue Date='2015-12-21T00:00:00', Issue time='1251', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='200304', Make='HOND', Body Style='PA', Color='GY', Location='13147 WELBY WAY', Route='01521', Agency='1', Violation code='4000A1', Violation Description='NO EVIDENCE OF REG', Fine amount='50')


## B.6 Find the maximum fine amount. How many fines have this amount?

In [7]:
# You need to convert the ‘fine amount’ column to a float to do this correctly.
# The maximum fine amount is 505, and there are 6 fines in this amount.

from pyspark.sql.types import DoubleType

df = parking_citations.withColumn("Fine amount", parking_citations['Fine amount'].cast(DoubleType()).alias('Fine amount'))
fine_a = df.select("Fine amount").na.fill(0)
maxi = fine_a.select('Fine amount').rdd.max()
print(f'The maximum fine amount is {maxi[0]}\n')

# Statistic for how many fines in the maximum amount
fine_a.groupBy('Fine amount').count().sort('Fine amount',ascending=False).show(5)


The maximum fine amount is 505.0

+-----------+-----+
|Fine amount|count|
+-----------+-----+
|      505.0|    6|
|      363.0|63366|
|      353.0|   15|
|      345.0|   40|
|      330.0|    1|
+-----------+-----+
only showing top 5 rows



## B.7 Show the top 20 most frequent vehicle makes, and their frequencies.

In [8]:
# The top 20 most frequent vehicle makes

print(f'Here are the most frequent vehicle makers and their frequencies:\n')

parking_citations.groupBy('Make').count().sort('count',ascending=False).show(20)

Here are the most frequent vehicle makers and their frequencies:

+----+-------+
|Make|  count|
+----+-------+
|TOYT|1531949|
|HOND|1043276|
|FORD| 807498|
|NISS| 662097|
|CHEV| 631413|
| BMW| 422916|
|MERZ| 376830|
|VOLK| 316002|
|HYUN| 285286|
|DODG| 271590|
|LEXS| 263269|
| KIA| 217795|
|JEEP| 214965|
|AUDI| 179718|
|MAZD| 169811|
|OTHR| 154376|
| GMC| 132788|
|INFI| 120340|
|CHRY| 120317|
|ACUR| 111265|
+----+-------+
only showing top 20 rows



## B.8 Expand some abbreviations in the color column. 

In [9]:
import pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

COLORS = {'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black',
          'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze',
          'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold',
          'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory',
          'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon',
          'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver',
          'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White',
          'WH':'White', 'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'}

def color(cl):
    if cl  not in COLORS.keys():
        return cl
    else:
        return COLORS[cl]
    
udf_color = udf(color,StringType())
cdf = parking_citations.withColumn('color long', udf_color('Color'))
print(f'Here is the head information afer add "color long" in the dataframe:\n{cdf.head()}')
cdf.select('color long').show(5)

Here is the head information afer add "color long" in the dataframe:
Row(Ticket number='1103341116', Issue Date='2015-12-21T00:00:00', Issue time='1251', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='200304', VIN=None, Make='HOND', Body Style='PA', Color='GY', Location='13147 WELBY WAY', Route='01521', Agency='1', Violation code='4000A1', Violation Description='NO EVIDENCE OF REG', Fine amount='50', Latitude='99999', Longitude='99999', color long='Gray')
+----------+
|color long|
+----------+
|      Gray|
|     White|
|     Black|
|     White|
|     Black|
+----------+
only showing top 5 rows



## B.9 Using this new column, what’s the most frequent colour value for Toyotas (TOYT)?

In [10]:
# The most frequent colour value for Toyotas (TOYT) is Gray.

reslut = cdf.filter("Make = 'TOYT'").groupBy('color long').count().sort('count',ascending=False)
reslut.show(10)

+----------+------+
|color long| count|
+----------+------+
|      Gray|346822|
|     White|304620|
|     Black|252199|
|    Silver|248685|
|      Blue|128051|
|       Red| 84175|
|     Green| 57627|
|      Gold| 30154|
|    Maroon| 19882|
|       Tan| 17006|
+----------+------+
only showing top 10 rows

